 # Proceso de carga de datos historicos

In [2]:
import pandas as pd
import logging
import os
from google.cloud import bigquery
from google.oauth2 import service_account

 # Configuración del sistema de excepciones y logs

In [3]:
# Crear un directorio de registros si no existe
log_dir = "logs"
os.makedirs(log_dir, exist_ok=True)

# Ajustar la configuración de registro
log_file = os.path.join(log_dir, "application_carga.log")
logging.basicConfig(
    level=logging.INFO, format="%(asctime)s - %(levelname)s - %(lineno)s - %(message)s"
)
file_handler = logging.FileHandler(log_file, "w")
file_handler.setLevel(logging.INFO)
file_handler.setFormatter(
    logging.Formatter("%(asctime)s - %(levelname)s - %(lineno)s - %(message)s")
)
logging.getLogger().addHandler(file_handler)

logging.debug("Prueba de mensaje de depuración")
logging.info("Prueba de mensaje informativo")
logging.warning("Prueba de mensaje de advertencia")
logging.error("Prueba de mensaje de error")

2023-05-21 21:34:49,314 - INFO - 18 - Prueba de mensaje informativo
2023-05-21 21:34:49,322 - WARNING - 19 - Prueba de mensaje de advertencia
2023-05-21 21:34:49,325 - ERROR - 20 - Prueba de mensaje de error


 Se carga el archivo parquet con la data limpia y se procede a dividir en varios dataframes para crear las diferentes tablas del modelo.

In [4]:
df = pd.read_parquet("datasets/flights_clean.parquet", engine="auto")
df_estados = pd.read_csv("datasets/us_states.tsv", sep="\t")

 Primero se extraen todos los aeropuertos, tanto origen como destino y se crea un solo dataframe con los valores únicos.

In [5]:
df_aeropuertos_origen = df[["ORIGINAIRPORTCODE",
                            "ORIGAIRPORTNAME", "ORIGINCITYNAME"]]
df_aeropuertos_origen = df_aeropuertos_origen.rename(
    columns={
        "ORIGINAIRPORTCODE": "AIRPORTCODE",
        "ORIGAIRPORTNAME": "AIRPORTNAME",
        "ORIGINCITYNAME": "AIRPORTCITY",
    }
)
df_aeropuertos_origen = df_aeropuertos_origen.drop_duplicates()
df_aeropuertos_destino = df[["DESTAIRPORTCODE",
                             "DESTAIRPORTNAME", "DESTCITYNAME"]]
df_aeropuertos_destino = df_aeropuertos_origen.rename(
    columns={
        "DESTAIRPORTCODE": "AIRPORTCODE",
        "DESTAIRPORTNAME": "AIRPORTNAME",
        "DESTCITYNAME": "AIRPORTCITY",
    }
)
df_aeropuertos_destino = df_aeropuertos_destino.drop_duplicates()
df_aeropuertos = pd.concat([df_aeropuertos_origen, df_aeropuertos_destino])
df_aeropuertos = df_aeropuertos.drop_duplicates()

 Se organiza el dataframe de estados

In [6]:
df_estados = df_estados[["state", "name"]]
df_estados = df_estados.rename(
    columns={"state": "STATECODE", "name": "STATENAME"})
df_estados = df_estados.drop_duplicates()

 Aerolineas

In [7]:
df_aerolineas = df[["AIRLINECODE", "AIRLINENAME"]]
df_aerolineas = df_aerolineas.drop_duplicates()

 Vuelos
 - Para efectos prácticos de la prueba no se subierán los registros del último año para subir estos por medio de la ETL

In [8]:
df_vuelos = df.drop(
    columns=[
        "ORIGAIRPORTNAME",
        "ORIGINCITYNAME",
        "DESTAIRPORTNAME",
        "DESTCITYNAME",
        "AIRLINENAME",
        "ORIGINSTATENAME",
        "DESTSTATENAME",
    ]
)
ultimo_ano = df_vuelos["FLIGHTDATE"].dt.year.max()
df_vuelos = df_vuelos[df_vuelos["FLIGHTDATE"].dt.year < ultimo_ano]
del df

 Se crea una función para subir datasets a Bigquery

In [9]:
json_gcp = "service_account.json"
credentials = service_account.Credentials.from_service_account_file(
    json_gcp, scopes=["https://www.googleapis.com/auth/cloud-platform"]
)
project = "pruebatec-387103"


def upload_dataframe_to_bigquery(dataframe, project, table_name):
    # Initialize BigQuery client
    client = bigquery.Client(credentials=credentials, project=project)

    # Create BigQuery load job configuration
    job_config = bigquery.LoadJobConfig(
        autodetect=True, write_disposition="WRITE_TRUNCATE"
    )

    # Start BigQuery load job
    job = client.load_table_from_dataframe(
        dataframe, table_name, job_config=job_config
    )  # Make an API request.
    job.result()  # Wait for the job to complete.

    table = client.get_table(table_name)  # Make an API request.
    print(
        "Loaded {} rows and {} columns to {}".format(
            table.num_rows, len(table.schema), table_name
        )
    )

 Se suben los dataframes a sus respectivas tablas

In [10]:
upload_dataframe_to_bigquery(
    df_aerolineas, project, "pruebatec-387103.vuelosnq.DIM_AEROLINEAS"
)
upload_dataframe_to_bigquery(
    df_aeropuertos, project, "pruebatec-387103.vuelosnq.DIM_AEROPUERTOS"
)
upload_dataframe_to_bigquery(
    df_vuelos, project, "pruebatec-387103.vuelosnq.FACT_VUELOS"
)

Loaded 26 rows and 2 columns to pruebatec-387103.vuelosnq.DIM_AEROLINEAS
Loaded 362 rows and 3 columns to pruebatec-387103.vuelosnq.DIM_AEROPUERTOS
Loaded 1091447 rows and 24 columns to pruebatec-387103.vuelosnq.FACT_VUELOS


 Con esto finaliza el proceso de carga de la información.